Загрузка датасета:

In [ ]:
!gdown 1P6t3MAyTlsdN6ZQ9dzWtkHimYVY44J26

Downloading...
From: https://drive.google.com/uc?id=1P6t3MAyTlsdN6ZQ9dzWtkHimYVY44J26
To: /content/videos.zip
100% 293M/293M [00:03<00:00, 85.9MB/s]


In [ ]:
!gdown 1ajNw5P-mcRtezZ8b37ad-4UMFW5uirnS
!gdown 1Wcliu4PZgChLtoG3_eMYaM03AYnrdA7u

Downloading...
From: https://drive.google.com/uc?id=1ajNw5P-mcRtezZ8b37ad-4UMFW5uirnS
To: /content/train.csv
100% 57.7k/57.7k [00:00<00:00, 84.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Wcliu4PZgChLtoG3_eMYaM03AYnrdA7u
To: /content/val.csv
100% 6.44k/6.44k [00:00<00:00, 31.0MB/s]


In [ ]:
!unzip videos.zip

Archive:  videos.zip
  inflating: videos/00001.mp4        
  inflating: videos/00002.mp4        
  inflating: videos/00003.mp4        
  inflating: videos/00004.mp4        
  inflating: videos/00005.mp4        
  inflating: videos/00007.mp4        
  inflating: videos/00008.mp4        
  inflating: videos/00009.mp4        
  inflating: videos/00010.mp4        
  inflating: videos/00011.mp4        
  inflating: videos/00012.mp4        
  inflating: videos/00013.mp4        
  inflating: videos/00014.mp4        
  inflating: videos/00015.mp4        
  inflating: videos/00016.mp4        
  inflating: videos/00017.mp4        
  inflating: videos/00019.mp4        
  inflating: videos/00020.mp4        
  inflating: videos/00022.mp4        
  inflating: videos/00024.mp4        
  inflating: videos/00025.mp4        
  inflating: videos/00027.mp4        
  inflating: videos/00028.mp4        
  inflating: videos/00030.mp4        
  inflating: videos/00031.mp4        
  inflating: videos/00033.mp4

Установка и импортирование зависимостей:

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import albumentations as A

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoProcessor, AutoModel

from sklearn.metrics import f1_score

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

Предобработка датасета:

In [ ]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

In [ ]:
train_df.head(10)

,video_id,label
0,01781.mp4,mosh pit dancing
1,01005.mp4,dancing gangnam style
2,00305.mp4,breakdancing
3,01385.mp4,dancing macarena
4,00493.mp4,country line dancing
5,02033.mp4,salsa dancing
6,01817.mp4,robot dancing
7,01143.mp4,dancing gangnam style
8,00233.mp4,breakdancing
9,02999.mp4,tap dancing


In [ ]:
classes = train_df.label.unique().tolist()
classes

['mosh pit dancing',
 'dancing gangnam style',
 'breakdancing',
 'dancing macarena',
 'country line dancing',
 'salsa dancing',
 'robot dancing',
 'tap dancing',
 'dancing ballet',
 'square dancing',
 'belly dancing',
 'jumpstyle dancing',
 'dancing charleston',
 'swing dancing',
 'tango dancing']

In [ ]:
classes_dict = {x: str(i) for i, x in enumerate(classes)}
classes_dict

{'mosh pit dancing': '0',
 'dancing gangnam style': '1',
 'breakdancing': '2',
 'dancing macarena': '3',
 'country line dancing': '4',
 'salsa dancing': '5',
 'robot dancing': '6',
 'tap dancing': '7',
 'dancing ballet': '8',
 'square dancing': '9',
 'belly dancing': '10',
 'jumpstyle dancing': '11',
 'dancing charleston': '12',
 'swing dancing': '13',
 'tango dancing': '14'}

In [ ]:
train_df['map_label'] = train_df['label'].map(classes_dict)
val_df['map_label'] = val_df['label'].map(classes_dict)

In [ ]:
train_df.head(10)

,video_id,label,map_label
0,01781.mp4,mosh pit dancing,0
1,01005.mp4,dancing gangnam style,1
2,00305.mp4,breakdancing,2
3,01385.mp4,dancing macarena,3
4,00493.mp4,country line dancing,4
5,02033.mp4,salsa dancing,5
6,01817.mp4,robot dancing,6
7,01143.mp4,dancing gangnam style,1
8,00233.mp4,breakdancing,2
9,02999.mp4,tap dancing,7


In [ ]:
train_df.label.nunique()

15

In [ ]:
!mkdir processed

In [ ]:
paths = sorted(os.listdir('videos'))
len(paths)

2379

In [ ]:
root = 'videos'
out_path = 'processed'

for vid in paths:
    check_path = os.path.join(out_path, vid.split('.')[0])
    if not os.path.exists(check_path):
        os.mkdir(check_path)
    cap = cv2.VideoCapture(os.path.join(root, vid))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    amount = min((int(length // 8), 5))
    i, j = 0, 0
    flag = True
    while flag:
        ret, frame = cap.read()
        if ret:
            if j % amount == 0:      
                frame = frame[:, :, ::-1]
                save_path = os.path.join(out_path, vid.split('.')[0], f'{i}' + '.jpg')
                plt.imsave(save_path, frame)
                last_frame = frame
                i += 1

            if i == 8:
                flag = False
            j += 1
        else:
            if i < 8:
                for _ in range(8 - i):
                    save_path = os.path.join(out_path, vid.split('.')[0], f'{i}' + '.jpg')
                    plt.imsave(save_path, last_frame)
                    i += 1                
                flag = False

In [ ]:
len(os.listdir('processed'))

2379

In [ ]:
train_ids = [(row.video_id.split('.')[0], int(row.map_label)) for row in train_df.itertuples()]
val_ids = [(row.video_id.split('.')[0], int(row.map_label)) for row in val_df.itertuples()]

In [ ]:
transform = A.Compose([    
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.5,
                               contrast_limit=0.5, p=0.5)
], additional_targets={
    f'image{i}': 'image'
    for i in range(1, 8)
})

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, ids, images_root, transform=None):
        self.ids = ids
        self.images_root = images_root
        self.transform = transform

    def load_sample(self, paths):
        images = []
        for path in paths:
            img = plt.imread(path)
            images.append(img)

        return np.stack(images)

    def apply_augmentations(self, array):
        targets={'image': array[0]}
        for i in range(1, 8):
            targets[f'image{i}'] = array[i]
        transformed = self.transform(**targets)
        transformed = np.concatenate(
            [np.expand_dims(transformed['image'], axis=0)] 
            + [np.expand_dims(transformed[f'image{i}'], axis=0) for i in range(1, 8)]
        )
        return transformed

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        dir, label = self.ids[index]
        # loading, transforming and stacking together
        paths = [os.path.join(self.images_root, dir, \
                              f'{str(x)}.jpg') for x in range(8)]
        
        video = self.load_sample(paths)
        if self.transform:
            video = self.apply_augmentations(video)

        inputs = processor(            
            text=[''],
            videos=list(video),
            return_tensors="pt",
            padding=True,
        )

        for i in inputs:
            inputs[i] = inputs[i][0]

        return inputs, label

In [ ]:
train_dataset = CustomDataset(
    ids=train_ids,
    images_root='processed',
    transform=transform
)

val_dataset = CustomDataset(
    ids=val_ids,
    images_root='processed'
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=15, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=15, shuffle=False, num_workers=2)

In [ ]:
model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")
model.to(device)
classifier = nn.Linear(512, len(classes))
classifier.to(device)

Linear(in_features=512, out_features=15, bias=True)

Обучение:

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
epochs = 5
freeze_epochs = 5
model_lr = 1e-5
classifier_lr = 1e-3

model_optimizer = torch.optim.AdamW(model.parameters(), model_lr)
classifier_optimizer = torch.optim.AdamW(classifier.parameters(), lr=classifier_lr)

criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):

    model.eval()
    classifier.train()     

    train_loss = []
    for i, (batch, targets) in enumerate(tqdm(train_loader, desc=f"Epoch: {epoch}")):
        classifier_optimizer.zero_grad()

        batch = batch.to(device)
        targets = targets.to(device)

        with torch.no_grad():
            outputs = model(**batch)
        logits = classifier(outputs.video_embeds)

        loss = criterion(logits, targets) 
        loss.backward()
        classifier_optimizer.step()

        train_loss.append(loss.item())

    print('Training loss:', np.mean(train_loss))

    model.eval() 
    classifier.eval()    

    val_loss = []
    val_targets = []
    val_preds = []
    for i, (batch, targets) in enumerate(tqdm(val_loader, desc=f"Epoch: {epoch}")):
        with torch.no_grad():

            batch = batch.to(device)
            targets = targets.to(device)

            outputs = model(**batch)
            logits = classifier(outputs.video_embeds)

            loss = criterion(logits, targets) 

            val_loss.append(loss.item())
            val_targets.extend(targets.cpu().numpy())
            val_preds.extend(logits.argmax(axis=1).cpu().numpy())

    print('Val loss:', np.mean(val_loss))
    print('F1:', f1_score(val_targets, val_preds, average='macro'))

Epoch: 0:   0%|          | 0/143 [00:00<?, ?it/s]

Training loss: 2.553196038399543


Epoch: 0:   0%|          | 0/16 [00:00<?, ?it/s]

Val loss: 2.359584927558899
F1: 0.5973221561769165


Epoch: 1:   0%|          | 0/143 [00:00<?, ?it/s]

...

Val loss: 1.684939481317997
F1: 0.6145948610194252


Epoch: 4:   0%|          | 0/143 [00:00<?, ?it/s]

Training loss: 1.7156003846988812


Epoch: 4:   0%|          | 0/16 [00:00<?, ?it/s]

Val loss: 1.553726702928543
F1: 0.6251955780916696


In [ ]:
for param in model.parameters():
    param.requires_grad = True
for param in model.text_model.parameters():
    param.requires_grad = False

In [ ]:
for epoch in range(epochs):

    model.train() 
    classifier.train()     

    train_loss = []
    for i, (batch, targets) in enumerate(tqdm(train_loader, desc=f"Epoch: {epoch}")):
        model_optimizer.zero_grad()
        classifier_optimizer.zero_grad()

        batch = batch.to(device)
        targets = targets.to(device)

        outputs = model(**batch)
        logits = classifier(outputs.video_embeds)

        loss = criterion(logits, targets) 
        loss.backward()
        model_optimizer.step()
        classifier_optimizer.step()

        train_loss.append(loss.item())

    print('Training loss:', np.mean(train_loss))

    model.eval()
    classifier.eval()   

    val_loss = []
    val_targets = []
    val_preds = []
    for i, (batch, targets) in enumerate(tqdm(val_loader, desc=f"Epoch: {epoch}")):
        with torch.no_grad():

            batch = batch.to(device)
            targets = targets.to(device)

            outputs = model(**batch)
            logits = classifier(outputs.video_embeds)

            loss = criterion(logits, targets) 

            val_loss.append(loss.item())
            val_targets.extend(targets.cpu().numpy())
            val_preds.extend(logits.argmax(axis=1).cpu().numpy())           

    print('Val loss:', np.mean(val_loss))
    print('F1:', f1_score(val_targets, val_preds, average='macro'))

Epoch: 0:   0%|          | 0/143 [00:00<?, ?it/s]

Training loss: 1.4951154706361411


Epoch: 0:   0%|          | 0/16 [00:00<?, ?it/s]

Val loss: 1.2399392127990723
F1: 0.627647303830156


Epoch: 1:   0%|          | 0/143 [00:00<?, ?it/s]

...

Val loss: 1.1337752267718315
F1: 0.5996066493137313


Epoch: 4:   0%|          | 0/143 [00:00<?, ?it/s]

Training loss: 0.45756081726167586


Epoch: 4:   0%|          | 0/16 [00:00<?, ?it/s]

Val loss: 1.3637132104486227
F1: 0.6024218609596663


Максимальное значение метрики F1(macro) - 0,66, в среднем - порядка 0,6.

На последней эпохе модель уже немного переобучилась.